<a href="https://colab.research.google.com/github/jwengr/dacon/blob/main/%EC%8B%9C%EC%8A%A4%ED%85%9C%20%ED%92%88%EC%A7%88%20%EB%B3%80%ED%99%94%EB%A1%9C%20%EC%9D%B8%ED%95%9C%20%EC%82%AC%EC%9A%A9%EC%9E%90%20%EB%B6%88%ED%8E%B8%20%EC%98%88%EC%A7%80%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!unzip drive/My\ Drive/dacon/lg/dataset/data.zip -d drive/My\ Drive/dacon/lg/dataset

In [ ]:
!pip install pandas_profiling --upgrade

In [ ]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#pd.options.mode.chained_assignment = None

In [ ]:
defaultpath = 'drive/My Drive/dacon/lg'

그냥 csv를 읽을시 데이터 손실이 나기때문에(타입문제) str타입으로 읽어준다.

In [ ]:
err_df= pd.read_csv(defaultpath+'/dataset/train_err_data.csv',encoding='utf-8',dtype=np.str)
quality_df= pd.read_csv(defaultpath+'/dataset/train_quality_data.csv',encoding='utf-8',dtype=np.str)
problem_df= pd.read_csv(defaultpath+'/dataset/train_problem_data.csv',encoding='utf-8',dtype=np.str)

test_err_df= pd.read_csv(defaultpath+'/dataset/test_err_data.csv',encoding='utf-8',dtype=np.str)
test_quality_df= pd.read_csv(defaultpath+'/dataset/test_quality_data.csv',encoding='utf-8',dtype=np.str)
sample_submission= pd.read_csv(defaultpath+'/dataset/sample_submission.csv',encoding='utf-8',dtype=np.str)

pandas profiling이 메모리문제로 안됨-> 일단 데이터를 직접 확인하자

In [ ]:
problem_df.head()

,user_id,time
0,19224,20201102200000
1,23664,20201116140000
2,15166,20201114130000
3,12590,20201108210000
4,15932,20201103210000


In [ ]:
problem_df.describe()

,user_id,time
count,5429,5429
unique,5000,488
top,24407,20201130120000
freq,5,42


In [ ]:
problem_df.isna().sum()

user_id    0
time       0
dtype: int64

In [ ]:
sample_submission.head()

,user_id,problem
0,30000,0
1,30001,0
2,30002,0
3,30003,0
4,30004,0


In [ ]:
sample_submission.describe()

,user_id,problem
count,14999,14999
unique,14999,1
top,31645,0
freq,1,14999


In [ ]:
err_df.describe()

,user_id,time,model_nm,fwver,errtype,errcode
count,16554663,16554663,16554663,16554663,16554663,16554662
unique,15000,2566163,9,37,41,2805
top,24934,20201108023257,model_1,04.16.3553,23,1
freq,222186,631,5384491,5237816,2276515,8906967


In [ ]:
err_df.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [ ]:
err_df.isna().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     1
dtype: int64

In [ ]:
quality_df.describe()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
count,828624,828624,788544,684192,828624,788511,828624,828624,828604,828624,828624,828624,828624,828624,828624,828624
unique,4319,8281,27,753,31,798,1,1,4744,549,884,42,523,4200,12,14
top,20201110143000,15357,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0
freq,948,4224,163236,542790,670270,632469,828624,828624,428096,662217,757788,787812,796284,99828,672229,801156


In [ ]:
quality_df.head()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


In [ ]:
quality_df.isna().sum()

time               0
user_id            0
fwver          40080
quality_0     144432
quality_1          0
quality_2      40113
quality_3          0
quality_4          0
quality_5         20
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

quality data의 nan 값이 상당히 많기 때문에 이를 잘 처리해야한다.

problem data : 5000명의 유저가 에러를 신고한 시간이다. 
5000명의 유저가 신고했지만 기간은 480에 불과하다.
즉 같은 시간대에 여러 유저들이 불편함을 느끼고 신고하였다.
sample submission : 14999 유저의 에러신고여부를 예측해야한다
err data : 15000명의 유저가 에러를 겪은 시간대와 그 에러종류, 모델, 버전정보가 있다. 한명의 유저가 여러시간대에 여러 에러를 겪는다.
quality data : 4319의 시간기간에 시스템 퀄리티의 정보와 사용한 유저가 있다.

problem data 분석

In [ ]:
problem_df['year'] = problem_df['time'].apply(lambda x: x[:4])
problem_df['month'] = problem_df['time'].apply(lambda x:  x[4:6])
problem_df['day'] = problem_df['time'].apply(lambda x: x[6:8])
problem_df['hour'] = problem_df['time'].apply(lambda x: x[8:10])
problem_df['mmss'] = problem_df['time'].apply(lambda x: x[10:]) 

In [ ]:
problem_df.describe()

,user_id,time,year,month,day,hour,mmss
count,5429,5429,5429,5429,5429,5429,5429
unique,5000,488,1,1,30,24,1
top,24407,20201130120000,2020,11,30,12,0000
freq,5,42,5429,5429,287,627,5429


problem data는 2020년 11월 한달동안의 에러를 겪은 유저이다.
즉 11월 한달동안 에러를 신고한 유저를 예측해야 한다.
mmss는 기록되지 않는다.  유저의 id구간은 10001~24998이다.
한 유저가 한달동안에 여러 에러를 겪어서 여러번 신고할 수 도 있지만, 여러번 신고하더라도 1번으로 기록된다.단순 0 1 정확도 분석보다는 여러 모델의 Cross entropy를 ensemble한 것이 좋을것 같다는 생각.

err data 분석

In [ ]:
err_df['year'] = err_df['time'].apply(lambda x: x[:4])
err_df['month'] = err_df['time'].apply(lambda x:  x[4:6])
err_df['day'] = err_df['time'].apply(lambda x: x[6:8])
err_df['hour'] = err_df['time'].apply(lambda x: x[8:10])
err_df['mmss'] = err_df['time'].apply(lambda x: x[10:]) 

In [ ]:
err_df.describe()

,user_id,time,model_nm,fwver,errtype,errcode,year,month,day,hour,mmss
count,16554663,16554663,16554663,16554663,16554663,16554662,16554663,16554663,16554663,16554663,16554663
unique,15000,2566163,9,37,41,2805,1,3,31,24,3600
top,24934,20201108023257,model_1,04.16.3553,23,1,2020,11,21,19,0008
freq,222186,631,5384491,5237816,2276515,8906967,16554663,16554577,649909,899350,5377


err data 도 2020년인건 동일하지만 11월을 전후로 3달간의 데이터입니다.

In [ ]:
err_df['time'].astype(int).describe().astype(int)

count          16554663
mean     20201115695081
std             8664375
min      20201031235959
25%      20201108122625
50%      20201116030018
75%      20201123135610
max      20201202185152
Name: time, dtype: int64

err data는 10월말부터 12월초까지의 데이터 입니다.
다른 날짜들의 데이터가 도움이 될까요?
일단 11월의 데이터만 살펴보겠습니다.

In [ ]:
err_df[err_df['time'].str.contains('202011.{8}')].describe()

,user_id,time,model_nm,fwver,errtype,errcode,year,month,day,hour,mmss
count,16554577,16554577,16554577,16554577,16554577,16554576,16554577,16554577,16554577,16554577,16554577
unique,15000,2566096,9,37,41,2805,1,1,30,24,3600
top,24934,20201108023257,model_1,04.16.3553,23,1,2020,11,21,19,0008
freq,222186,631,5384470,5237795,2276513,8906919,16554577,16554577,649909,899349,5377


err_df에서
11월 총 유저는 16554577 이고 중복을 제거하면
15000명의 유저가 에러를 겪었다.
그중 5천명이 에러를 신고하였습니다.

각 유저들의 에러 경험 수를 컬럼으로 만들고 완전중복인 행을 제거합니다.

한달동안의 데이터가 있고 이 데이터를 0 ,1 로 분류하는 방법을 설계해 봅시다.
1.시계열 분석(rnn 히든마르코프 등) 딥러닝, 머신러닝
2.한달의 데이터의 요약 : 고전적, 머신러닝, 금융공학
3.완전중복만 제거 후 머신러닝(기본)

10월 말과 12월초의 데이터에 0,1을 부여하여 학습에 사용 할  수도 있습니다->몬테카를로(시계열),머신러닝 등

quality data 분석

In [ ]:
quality_df['year'] = quality_df['time'].apply(lambda x: x[:4])
quality_df['month'] = quality_df['time'].apply(lambda x:  x[4:6])
quality_df['day'] = quality_df['time'].apply(lambda x: x[6:8])
quality_df['hour'] = quality_df['time'].apply(lambda x: x[8:10])
quality_df['mmss'] = quality_df['time'].apply(lambda x: x[10:]) 

In [ ]:
quality_df.describe()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,year,month,day,hour,mmss
count,828624,828624,788544,684192,828624,788511,828624,828624,828604,828624,828624,828624,828624,828624,828624,828624,828624,828624,828624,828624,828624
unique,4319,8281,27,753,31,798,1,1,4744,549,884,42,523,4200,12,14,1,2,31,24,6
top,20201110143000,15357,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0,2020,11,08,22,2000
freq,948,4224,163236,542790,670270,632469,828624,828624,428096,662217,757788,787812,796284,99828,672229,801156,828624,828408,33996,59352,151224


In [ ]:
quality_df['time'].astype(int).describe().astype(int)

count            828624
mean     20201115421121
std             8863638
min      20201031235000
25%      20201108102750
50%      20201115182500
75%      20201123131000
max      20201130234000
Name: time, dtype: int64

quality data는 2020년 10월 11일부터 2020년 11월 30일 23시까지의 로그를 가지고 있다.

In [ ]:
quality_ppdf = quality_df[quality_df['time'].str.contains('202011.{8}')].drop(['time','year','month','day','mmss'],axis=1)

In [ ]:
quality_ppdf.isna().sum()

user_id            0
fwver          40068
quality_0     144420
quality_1          0
quality_2      40101
quality_3          0
quality_4          0
quality_5         20
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
hour               0
dtype: int64

nan 값이 발생한 row만 살펴보겠습니다.

In [ ]:
quality_df[quality_df.isna().sum(axis=1)>0].describe()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,year,month,day,hour,mmss
count,144485,144485,104405,53,144485,104372,144485,144485,144465,144485,144485,144485,144485,144485,144485,144485,144485,144485,144485,144485,144485
unique,3913,587,7,2,27,120,1,1,725,204,241,25,83,597,10,11,1,2,31,24,6
top,20201104204000,23065,03.11.1167,0.0,0,0.0,0,0,0,0,0,0,0,1,0,0,2020,11,27,22,2000
freq,180,3828,104148,52,120844,89734,144485,144485,100578,118698,128569,136013,137912,39600,119411,121847,144485,144473,5681,10213,27530


특정 fwver과 quality_0에서의 nan이 주목할만 합니다.

In [ ]:
quality_ppdf.describe()

,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,hour
count,828408,788340,683988,828408,788307,828408,828408,828388,828408,828408,828408,828408,828408,828408,828408,828408
unique,8281,27,753,31,798,1,1,4743,549,884,42,523,4198,12,14,24
top,15357,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0,22
freq,4224,163176,542614,670084,632293,828408,828408,427965,662035,757608,787608,796068,99816,672042,800940,59352


In [ ]:
for i in range(13):
    print('quality_'+str(i),set(quality_ppdf[quality_ppdf.notna()].drop(['user_id','fwver'],axis=1).iloc[:,i].values))

quality_0 {nan, '197.0', '127.0', '207.0', '386.0', '404.0', '77.0', '13.0', '28.0', '422.0', '267.0', '66.0', '1488.0', '4492.0', '408.0', '259.0', '186.0', '695.0', '11284.0', '18953.0', '234.0', '566.0', '488.0', '97.0', '6964.0', '1243.0', '332.0', '135.0', '570.0', '357.0', '882.0', '15.0', '383.0', '8.0', '295.0', '72.0', '380.0', '292.0', '221.0', '458.0', '613.0', '9.0', '6975.0', '1027.0', '1310.0', '313.0', '1141.0', '2075.0', '101.0', '691.0', '442.0', '514.0', '780.0', '3187.0', '589.0', '3.0', '14562.0', '212.0', '296.0', '598.0', '261.0', '513.0', '889.0', '678.0', '4599.0', '453.0', '90.0', '4.0', '307.0', '233.0', '27.0', '949.0', '685.0', '44.0', '79.0', '87.0', '1007.0', '298.0', '1217.0', '188.0', '637.0', '624.0', '323.0', '1167.0', '178.0', '1038.0', '2351.0', '4973.0', '297.0', '4597.0', '888.0', '303.0', '222.0', '167.0', '18.0', '284.0', '374.0', '290.0', '143.0', '1851.0', '11356.0', '228.0', '492.0', '4970.0', '324.0', '302.0', '304.0', '4552.0', '41.0', '4324

quality data는 ,가 포함된 숫자이므로 숫자형으로 바꿔주어야한다.
quality 3,4 는 모든컬럼이 0이다 -> 삭제
대부분의 컬럼은 0이 제일 많다 -> quality 10은 예외
nan값과 -1값은 무엇을 뜻하는걸까?
unique값이 작은 데이터들은 범주형 데이터일까? -> test데이터와의 비교 필요

quality데이터와 err데이터의 비교
quality 유저수 8281 err 유저수 15000
quality fwver 27개 err fwver 37
quality 기간 20201011 - 20201130 err 기간 20201031 -20201202

In [ ]:
test_err_df['year'] = test_err_df['time'].apply(lambda x: x[:4])
test_err_df['month'] = test_err_df['time'].apply(lambda x:  x[4:6])
test_err_df['day'] = test_err_df['time'].apply(lambda x: x[6:8])
test_err_df['hour'] = test_err_df['time'].apply(lambda x: x[8:10])
test_err_df['mmss'] = test_err_df['time'].apply(lambda x: x[10:]) 

In [ ]:
test_err_df.describe()

,user_id,time,model_nm,fwver,errtype,errcode,year,month,day,hour,mmss
count,16532648,16532648,16532648,16532648,16532648,16532644,16532648,16532648,16532648,16532648,16532648
unique,14998,2567716,9,40,41,2955,1,3,31,24,3600
top,41285,20201129045433,model_1,04.16.3553,23,1,2020,11,12,19,0905
freq,396478,618,5473847,5326124,2460484,8750154,16532648,16532572,614600,898253,5477


In [ ]:
test_err_df['time'].astype(int).describe().astype(int)

count          16532648
mean     20201115490295
std             8673027
min      20201031235959
25%      20201108093640
50%      20201115174401
75%      20201123082358
max      20201214120118
Name: time, dtype: int64

test_err의 시작날짜는 1031로 동일하지만
err의 마지막날찌는 1202인 반면에 test_err은 1214에 끝납니다.
fwver또한 test_err에서는 40개로 새롭게 더 등장합니다.
errtype과 errcode또한 서로 다를지 확인할 필요가 있습니다.

In [ ]:
test_err_df[test_err_df['time'].str.contains('202011.{8}')].describe()

,user_id,time,model_nm,fwver,errtype,errcode,year,month,day,hour,mmss
count,16532572,16532572,16532572,16532572,16532572,16532568,16532572,16532572,16532572,16532572,16532572
unique,14998,2567648,9,40,41,2955,1,1,30,24,3600
top,41285,20201129045433,model_1,04.16.3553,23,1,2020,11,12,19,0905
freq,396478,618,5473839,5326116,2460483,8750122,16532572,16532572,614600,898253,5477


test quality도 분석해 봅시다

In [ ]:
test_quality_df['year'] = test_quality_df['time'].apply(lambda x: x[:4])
test_quality_df['month'] = test_quality_df['time'].apply(lambda x:  x[4:6])
test_quality_df['day'] = test_quality_df['time'].apply(lambda x: x[6:8])
test_quality_df['hour'] = test_quality_df['time'].apply(lambda x: x[8:10])
test_quality_df['mmss'] = test_quality_df['time'].apply(lambda x: x[10:]) 

In [ ]:
test_quality_df.describe()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,year,month,day,hour,mmss
count,747972,747972,725208,641388,747961,726857,747972,747972,747928,747972,747972,747972,747972,747972,747972,747972,747972,747972,747972,747972,747972
unique,4317,8268,22,540,34,596,1,1,3994,528,839,44,431,3515,13,17,1,2,31,24,6
top,20201110141000,37632,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0,2020,11,01,22,1000
freq,984,3792,142704,505710,601265,578838,747972,747972,399032,594594,682584,707868,715716,98688,603983,727188,747972,747840,31680,58344,144252


In [ ]:
test_quality_df['time'].astype(int).describe().astype(int)

count            747972
mean     20201115378414
std             8818205
min      20201031235000
25%      20201108104000
50%      20201115153000
75%      20201123112000
max      20201130234000
Name: time, dtype: int64

test_quality 데이터는 20201031에서 시작하여 20201130에 끝납니다.
quality 데이터는20201011에서 시작하여20201030에 끝납니다. test_quality데이터가 더 작음

In [ ]:
test_quality_df.isna().sum()

time               0
user_id            0
fwver          22764
quality_0     106584
quality_1         11
quality_2      21115
quality_3          0
quality_4          0
quality_5         44
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
year               0
month              0
day                0
hour               0
mmss               0
dtype: int64

test quality 데이터에도 nan값이 많이 있습니다.

In [ ]:
test_quality_df[test_quality_df.isna().sum(axis=1)>0].describe()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,year,month,day,hour,mmss
count,108267,108267,85503,1683,108256,87152,108267,108267,108223,108267,108267,108267,108267,108267,108267,108267,108267,108267,108267,108267,108267
unique,3555,579,7,15,30,145,1,1,550,119,129,29,122,432,8,10,1,1,30,24,6
top,20201102203000,32690,03.11.1167,0.0,0,0.0,0,0,0,0,0,0,0,1,0,0,2020,11,08,22,1000
freq,132,3408,84240,1267,92305,74399,108267,108267,79302,91254,98549,102422,98756,37488,91262,91403,108267,108267,4226,8353,21038


nan값이 있는 row만 확인해보았습니다.
특이점으로는 fwver도 총 22개지만 nan값이 발생한 fwver은 7개뿐이었습니다.
quality 데이터와의 유니크 값을 비교해보겠습니다.

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,year,month,day,hour,mmss
count,144485,144485,104405,53,144485,104372,144485,144485,144465,144485,144485,144485,144485,144485,144485,144485,144485,144485,144485,144485,144485
unique,3913,587,7,2,27,120,1,1,725,204,241,25,83,597,10,11,1,2,31,24,6
top,20201104204000,23065,03.11.1167,0.0,0,0.0,0,0,0,0,0,0,0,1,0,0,2020,11,27,22,2000
freq,180,3828,104148,52,120844,89734,144485,144485,100578,118698,128569,136013,137912,39600,119411,121847,144485,144473,5681,10213,27530
